# Libraries

In [19]:
import json, os, time, glob

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from tqdm.notebook import tqdm

import pandas as pd

# Parameters

In [2]:
base_url = 'https://www.tokopedia.com'
initial_url = 'https://www.tokopedia.com/find/kulkas-2-pintu'

# Buka Browser

In [3]:
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
#driver = webdriver.Chrome()

In [4]:
driver.get(initial_url)

In [11]:
driver.get('https://tokopedia.com/find/kulkas-2-pintu?page=2')

In [29]:
driver.get('https://shopee.co.id')

# Get Products' Element

In [12]:
elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

In [13]:
len(elements)

10

In [152]:
elements[0].text

'Ad\nDisplay Cooler RSA Topaz 600 / Topaz600 Kulkas Showcase 2 Pintu 568 L\nRp8.199.000\n4.9\n28 terjual\nRefcon Polar Nusaindo'

# Scroll the page

In [14]:
last_height = driver.execute_script("return document.body.scrollHeight")

In [15]:
while True:
    for i in range(30):
        driver.execute_script(f"window.scrollBy(0, 200);")
        #driver.execute_script(f"window.scrollBy(0, {i*last_height/20});")

        time.sleep(0.5)

    #time.sleep(3)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height >= last_height:
        break
    last_height = new_height

In [16]:
elements = driver.find_elements(By.XPATH, '//div[contains(@data-ssr, "findProductSSR")]')
#elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

In [17]:
len(elements)

80

In [20]:
li = []
pbar = tqdm(total=len(elements))
for element in elements:
    divs = element.find_elements(By.XPATH, './a/div/div[2]/div')
    name = element.find_element(By.XPATH, './a/div/div[2]/div[1]/span').text.strip()
    price = element.find_element(By.XPATH, './a/div/div[2]/div[2]/div').text.strip()

    shop_name = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[1]').text.strip()

    shop = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]')
    actions = ActionChains(driver)
    actions.move_to_element(shop).perform()

    shop_city = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[2]').text.strip()
    url = element.find_element(By.XPATH, './/a').get_attribute('href')

    li.append({
        'name': name,
        'price': price,
        'url': url,
        'shop_name': shop_name,
        'shop_city': shop_city
    })
    pbar.update(1)

df = pd.DataFrame(li)

pbar.close()

  0%|          | 0/80 [00:00<?, ?it/s]

In [21]:
df.head(20)

,name,price,url,shop_name,shop_city
0,KULKAS 2 PINTU SHARP SJ-195MN-HS NO FROST,Rp2.625.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Toko PROGRESS,Kab. Bogor
1,Kulkas 2 Pintu Aqua Sanyo D251 Tanpa Bunga Es ...,Rp2.580.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Maju Jaya Electronik,Jakarta Barat
2,LG GN-B312PQCP KULKAS 2 PINTU INVERTER 315 Lit...,Rp6.599.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,CT Electronics,Tangerang
3,Changhong Kulkas 2 Pintu Refrigerator Kapasita...,Rp2.369.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,changhong surabaya,Surabaya
4,Samsung Kulkas 2 Pintu w/ Ungkep compartment 3...,Rp7.969.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Samsung Exclusive Official Store,Kab. Tangerang
5,Timer Defrost Bimetal fuse kulkas AQUA 2 pintu...,Rp63.100,https://www.tokopedia.com/gsparepart-4/timer-d...,Gudang sparepart jkt,Jakarta Barat
6,Panasonic NR-BB201Q-PK Kulkas 2 Pintu + Invert...,Rp4.201.000,https://www.tokopedia.com/panasonic-official/p...,Panasonic,Bekasi
7,KULKAS AQUA JAPAN AQR-D275 INVERTER LEMARI ES ...,Rp3.119.000,https://www.tokopedia.com/topaircon/kulkas-aqu...,TOP AIRCON,Tangerang Selatan
8,LG GN-B215SQMT Kulkas 2 Pintu - Smart Inverter...,Rp3.750.000,https://www.tokopedia.com/getonlines/lg-gn-b21...,GET Online Shop,Jakarta Pusat
9,Panasonic NRBB200VH Kulkas 2 Pintu NRBB200VH 1...,Rp3.900.000,https://www.tokopedia.com/setia-makmuroff/pana...,Setia Makmur Elektronik,Jakarta Barat


# Create Dir

In [22]:
dir_path = 'scrape_tokopedia_selenium'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [24]:
files = glob.glob(os.path.join(dir_path, '*'))
for file in files:
    try:
        os.remove(file)
    except: pass

# Scrape Multiple Pages

In [25]:
start_page = 1
end_page = 2
pbar = tqdm(total=end_page-start_page+1)

for page in range(start_page, end_page+1):
    url_to = initial_url + f'?page={page}'
    driver.get(url_to)

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(30):
            driver.execute_script(f"window.scrollBy(0, 200);")

            time.sleep(0.1)

        #time.sleep(3)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height >= last_height:
            break
        last_height = new_height

    elements = driver.find_elements(By.XPATH, '//div[contains(@data-ssr, "findProductSSR")]')
    #elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

    li = []
    for element in elements:
        divs = element.find_elements(By.XPATH, './a/div/div[2]/div')
        name = element.find_element(By.XPATH, './a/div/div[2]/div[1]/span').text.strip()
        price = element.find_element(By.XPATH, './a/div/div[2]/div[2]/div').text.strip()

        shop_name = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[1]').text.strip()

        shop = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]')
        actions = ActionChains(driver)
        actions.move_to_element(shop).perform()

        shop_city = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[2]').text.strip()
        url = element.find_element(By.XPATH, './/a').get_attribute('href')

        li.append({
            'name': name,
            'price': price,
            'url': url,
            'shop_name': shop_name,
            'shop_city': shop_city
        })

    df = pd.DataFrame(li)
    df.to_csv(os.path.join(dir_path, f'{page}.csv'), index=False, sep='\t')

    pbar.update(1)

pbar.close()

  0%|          | 0/2 [00:00<?, ?it/s]

# Combine All CSVs

In [26]:
all_files = os.listdir(dir_path)

csv_files = [file for file in all_files if file.endswith('.csv')]

In [27]:
li = []

for file in csv_files:
    df = pd.read_csv(os.path.join(dir_path, file), sep='\t')
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.to_excel(os.path.join(dir_path, 'hasil_scrape.xlsx'), index=False)

In [28]:
df

,name,price,url,shop_name,shop_city
0,GEA G2HD-267WD Home Refrigerator / Kulkas 2 Pi...,Rp3.720.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Dwi Karya Mesin,Surabaya
1,Sharp Kulkas 2 Pintu SJ-237MG-DB Magneglass Da...,Rp3.049.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,SHARP OFFICIAL STORE JAKARTA,Jakarta Utara
2,Changhong Kulkas 2 Pintu Refrigerator Kapasit...,Rp2.049.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,changhong surabaya,Surabaya
3,Kulkas 2 Pintu SHARP SJ 195 MD SG SR 200L Low ...,Rp2.629.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Dunia Electronik 12,Jakarta Barat
4,SAMSUNG Kulkas 2 Pintu - RT19M300BGS / 19M300B...,Rp3.700.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,LM Electronic,Bogor
...,...,...,...,...,...
155,SMEG FAB30 Free Standing Refrigerator Double D...,Rp47.500.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Toffin Indonesia Official Store,Jakarta Barat
156,kulkas Aqua 2 pintu inverter Glass Door 220 L...,Rp2.900.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,ANITA ELECTRONIK,Jakarta Barat
157,KULKAS 2 PINTU AQUA AQR-D261 AQRD 261 LOW WATT...,Rp2.879.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Bestprice Official Store,Jakarta Timur
158,Changhong Kulkas 2 Pintu Refrigerator Kapasita...,Rp2.369.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,changhong surabaya,Surabaya


# Close the browser

In [224]:
#driver.quit()